# Using EcoFOCIpy to process raw field data

## DY2407 (Spring Icthy - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2024/CTDcasts/dy2407/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2407' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2407.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2024/CTDcasts/dy2407/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2024/CTDcasts/dy2407/rawconverted/ctd002.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:91: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sal11,sal00,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sbeox1ml/l,sbeox1ps,sbox1mm/kg,sigma-t00,sigma-t11,c0ms/cm,c1ms/cm,fleco-afl,t090c,t190c,turbwetntu0,par,scan,prdm,datetime
count,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000e+00,7.000000,7.000000,7
mean,31.955214,31.956229,8.834029,114.741571,384.716000,8.662300,112.510571,377.236429,25.507500,25.506343,28.640380,28.643187,5.915300,2.347514,2.351914,0.576514,5.288286e-02,12607.000000,31.185286,2024-05-20 03:58:53.142857472
min,31.945100,31.944500,8.069100,103.841000,351.390000,7.919100,101.903000,344.858000,25.461100,25.460000,28.376878,28.375223,2.028900,1.998300,1.998100,0.441800,1.000000e-12,5318.000000,1.951000,2024-05-20 03:53:49
25%,31.946650,31.947850,8.142100,104.831000,354.572000,7.990550,102.883500,347.971500,25.483850,25.482750,28.392815,28.391461,2.891350,2.022250,2.022100,0.501750,1.000000e-12,9104.000000,14.766500,2024-05-20 03:56:27
50%,31.949400,31.949700,8.914300,115.908000,388.216000,8.752000,113.798000,381.146000,25.497700,25.496800,28.689854,28.697016,7.321100,2.416200,2.425200,0.586700,1.000000e-12,13028.000000,29.163000,2024-05-20 03:59:11
75%,31.965600,31.967450,9.337600,121.846500,406.652500,9.121050,119.019500,397.221500,25.541050,25.539600,28.796962,28.805945,8.270100,2.558550,2.569700,0.594850,1.000000e-12,16254.500000,44.978500,2024-05-20 04:01:25.500000
max,31.967500,31.968800,9.895400,130.087000,430.957000,9.741800,128.067000,424.265000,25.543900,25.542900,29.036375,29.035257,9.734200,2.856500,2.856500,0.813900,3.701800e-01,19186.000000,67.693000,2024-05-20 04:03:27
std,0.010424,0.011062,0.738132,10.566304,32.157336,0.717015,10.276799,31.237014,0.034236,0.034136,0.259719,0.262879,3.200862,0.340238,0.343711,0.122898,1.399149e-01,5027.699507,23.174459,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2407',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2024, 5, 18),
 'EndDate': datetime.date(2024, 5, 31),
 'Project': 'FOCI',
 'ChiefScientist': 'Kelia Axler',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Icthy Survey',
 'CruiseYear': 2024,
 'ctdlogs_pdf_name': ''}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54813,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2407',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2407',
 'Project': 'Spring Ichthy Survey',
 'StationNo_altname': '',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 56,
 'LatitudeMin': 52.1,
 'LongitudeDeg': 164,
 'LongitudeMin': 3.0,
 'GMTDay': 20,
 'GMTMonth': 'May',
 'GMTYear': 2024,
 'GMTTime': 42780,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': 1012,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 117,
 'WindSpd': 13.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 72,
 'StationNameID': 'M2',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '1;2;3;4;5;6;7',
 'NutrientBtlNumbers': '1;2;3;4;5;6;7',
 'OxygenBtlNiskinNo': '1;2;3;4;5;6;7',
 'OxygenBtlNumbers': '352;037;305;351;266;282;115',
 'SalinityBtlNiskinNo': '1',
 'SalinityBtlNumbers': '121',
 'ChlorophyllBtlNiskinNo': '1;2;3;4;5;6;7',
 'ChlorophyllBtlVolu

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,salinity_ch2,salinity_ch1,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,sigma_t_ch1,sigma_t_ch2,...,c1ms/cm,chlor_fluorescence,temperature_ch1,temperature_ch2,turbidity,par,scan,Pressure [dbar],datetime,cast
bottle,,,,,,,,,,,,,,,,,,,,,
2.0,31.9667,31.9688,8.0691,103.841,351.39,7.9492,102.298,346.169,25.5436,25.5419,...,28.375223,2.0289,2.0015,2.0015,0.4565,1.000000e-12,8144.0,50.388,2024-05-20 03:55:47,ctd001


In [10]:
cruise_data['ctd001.btl'].columns

Index(['salinity_ch2', 'salinity_ch1', 'oxy_concM_ch1', 'oxy_percentsat_ch1',
       'oxy_conc_ch1', 'oxy_concM_ch2', 'oxy_percentsat_ch2', 'oxy_conc_ch2',
       'sigma_t_ch1', 'sigma_t_ch2', 'c0ms/cm', 'c1ms/cm',
       'chlor_fluorescence', 'temperature_ch1', 'temperature_ch2', 'turbidity',
       'par', 'scan', 'Pressure [dbar]', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (bottle: 7)
Coordinates:
  * bottle              (bottle) float64 56B 1.0 2.0 3.0 4.0 5.0 6.0 7.0
Data variables: (12/21)
    salinity_ch2        (bottle) float64 56B 31.97 31.97 31.96 ... 31.95 31.95
    salinity_ch1        (bottle) float64 56B 31.97 31.97 31.97 ... 31.94 31.95
    oxy_concM_ch1       (bottle) float64 56B 8.086 8.069 8.198 ... 9.538 9.895
    oxy_percentsat_ch1  (bottle) float64 56B 104.1 103.8 105.6 ... 124.6 130.1
    oxy_conc_ch1        (bottle) float64 56B 352.1 351.4 357.0 ... 415.4 431.0
    oxy_concM_ch2       (bottle) float64 56B 7.919 7.949 8.032 ... 9.248 9.742
    ...                  ...
    turbidity           (bottle) float64 56B 0.8139 0.4565 ... 0.6009 0.5867
    par                 (bottle) float64 56B 1e-12 1e-12 1e-12 ... 1e-12 0.3702
    scan                (bottle) float64 56B 5.318e+03 8.144e+03 ... 1.919e+04
    Pressure [dbar]     (bottle) float64 56B 67.69 50.39 39.57 ... 9.638 1.951
    datetime            (bottle) datetime64[ns] 56B 2024-05-20T03:53:49 ... 2...
    cast                (bottle) object 56B 'ctd001' 'ctd001' ... 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 7)
Coordinates:
  * bottle              (bottle) float64 56B 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/21)
    salinity_ch2        (latitude, longitude, time, bottle) float64 56B 31.97...
    salinity_ch1        (latitude, longitude, time, bottle) float64 56B 31.97...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 56B 8.086...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 56B 104.1...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 56B 352.1...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 56B 7.919...
    ...                  ...
    turbidity           (latitude, longitude, time, bottle) float64 56B 0.813...
    par                 (latitude, longitude, time, bottle) float64 56B 1e-12...
    scan                (latitude, longitude, time, bottle) float64 56B 5.318...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 56B 67.69...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 56B ...
    cast                (latitude, longitude, time, bottle) object 56B 'ctd00...

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset> Size: 1kB
Dimensions:             (latitude: 1, longitude: 1, time: 1, bottle: 7)
Coordinates:
  * bottle              (bottle) float64 56B 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * latitude            (latitude) float64 8B 1e+35
  * longitude           (longitude) float64 8B 1e+35
  * time                (time) float64 8B 1e+35
Data variables: (12/21)
    salinity_ch2        (latitude, longitude, time, bottle) float64 56B 31.97...
    salinity_ch1        (latitude, longitude, time, bottle) float64 56B 31.97...
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 56B 8.086...
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 56B 104.1...
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 56B 352.1...
    oxy_concM_ch2       (latitude, longitude, time, bottle) float64 56B 7.919...
    ...                  ...
    turbidity           (latitude, longitude, time, bottle) float64 56B 0.813...
    par                 (latitude, longitude, time, bottle) float64 56B 1e-12...
    scan                (latitude, longitude, time, bottle) float64 56B 5.318...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 56B 67.69...
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 56B ...
    cast                (latitude, longitude, time, bottle) object 56B 'ctd00...
Attributes: (12/44)
    CruiseID:                      DY2407
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-10-17T10:26:27Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [19]:
cruise_data[cast] = cruise_data[cast].rename(columns={
                    't090c':'temperature_ch1',
                    't190c':'temperature_ch2',
                    'sal00':'salinity_ch1',
                    'sal11':'salinity_ch2',
                    'sbox0mm/kg':'oxy_conc_ch1',
                    'sbeox0ml/l':'oxy_concM_ch1',
                    'sbox1mm/kg':'oxy_conc_ch2',
                    'sbeox1ml/l':'oxy_concM_ch2',
                    'sbeox0ps':'oxy_percentsat_ch1',
                    'sbeox1ps':'oxy_percentsat_ch2',
                    'sigma-t00':'sigma_t_ch1',
                    'sigma-t11':'sigma_t_ch2',
                    'cstarat0':'Attenuation',
                    'cstartr0':'Transmittance',
                    'fleco-afl':'chlor_fluorescence',
                    'turbwetntu0':'turbidity',
                    'prdm':'pressure',
                    'empty':'empty', #this will be ignored
                    'flag':'flag'})

cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                            instrument_yaml=inst_config, 
                            operation_yaml=cruise_config,
                            operation_type='ctd')

cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

#add global attributes
cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cast_label = cast.lower().split('d')[-1].split('.')[0]
cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                           filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


ValueError: Forward slashes '/' are not allowed in variable and dimension names (got 'c0ms/cm'). Forward slashes are used as hierarchy-separators for HDF5-based files ('netcdf4'/'h5netcdf').

In [20]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'minutes' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})


## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history